In [55]:
#Para crear y análizar redes
import networkx as nx
#Para leer desde un archivo
import pandas as pd 
#Para generar errores aleatorios
import random
# using datetime module
import datetime
import os

In [16]:
def imprimir_hora():
    # ct stores current time
    ct = datetime.datetime.now()
    print("current time:-", ct)

In [56]:
class Red:
    #inicializar los atributos 
    def __init__(self, n, t, na):
        self.nombre = n
        self.topologia = t
        self.num_ataques = na

        # NÚMERO DE COMPONENTES CONECTADOS
        self.ncc_aleatorio = list()
        self.ncc_degree = list()
        self.ncc_closeness = list()
        self.ncc_harmonic = list()
        self.ncc_pagerank = list()
        self.ncc_betweenness = list()

        # PROPORCIÓN DEL COMPONENTE GIGANTE
        self.pcg_aleatorio = list()
        self.pcg_degree = list()
        self.pcg_closeness = list()
        self.pcg_harmonic = list()
        self.pcg_pagerank = list()
        self.pcg_betweenness = list()

        # DIÁMETRO
        self.diametro_aletorio = list()
        self.diametro_degree = list()
        self.diametro_closeness = list()
        self.diametro_harmonic = list()
        self.diametro_pagerank = list()
        self.diametro_betweenness = list()

        # LONGITUD PROMEDIO DE LA TRAYECTORIA
        self.aspl_aleatorio = list()
        self.aspl_degree = list()
        self.aspl_closeness = list()
        self.aspl_harmonic = list()
        self.aspl_pagerank = list()
        self.aspl_betweenness = list()

        # PROBABILIDAD DE CONEXIÓN
        self.pc_aleatorio = list()
        self.pc_degree = list()
        self.pc_closeness = list()
        self.pc_harmonic = list()
        self.pc_pagerank = list()
        self.pc_betweenness = list()

        # NODO ELIMINADO
        self.nodo_aleatorio = list()
        self.nodo_degree = list()
        self.nodo_closeness = list()
        self.nodo_harmonic = list()
        self.nodo_pagerank = list()
        self.nodo_betweenness = list()

    
    ### M E T R I C A S ###

    def num_componentes_conectados(self, G):
        #A la variable num_cc se le asigna el valor de componentes conectados.
        print("Componentes conectados")
        imprimir_hora()
        num_cc = nx.number_connected_components(G)
        return num_cc
    
    def prop_componente_gigante(self, G):
        print("Proporcion del componente gigagnte")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True) # Duda, por qué los ordena?
        G0 = G.subgraph(componentes_conectados[0]) #Componente Gigant    
        tcg = len(G0) #Tamaño del componente Gigante, cuenta los elementos de la lista G0
        n = len(nx.nodes(G)) # Cuenta el numero de nodos que tiene la red G
        pcg = tcg / n #Proporción del componente gigante
        return pcg

    def diametro(self, G):
        print("Diametro")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True)
        G0 = G.subgraph(componentes_conectados[0])
        diameter = nx.diameter(G0)
        return diameter

    def trayectoria_promedio(self, G):
        print("Trayectoria promedio")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True)
        G0 = G.subgraph(componentes_conectados[0])
        aspl = nx.average_shortest_path_length(G0) #longitud de ruta más corta promedio
        return aspl

    def probabilidad_de_conexion(self, G):
        print("Probabilidad de conexión")
        imprimir_hora()
        componentes = sorted(nx.connected_components(G))
        sumar_componentes = []
        sumar_probabilidad = []
        for x in componentes:
            y = (len(x))
            sumar_componentes.append(y)
            sumar_probabilidad.append((y * (y - 1))/2)
        resultado = sum(sumar_componentes)
        if resultado > 1:
            probabilidad = sum(sumar_probabilidad) / ((resultado * (resultado - 1))/ 2)
        else:
            probabilidad = 0
        return probabilidad

    
    ### A T A Q U E S ###    
    def centrality(self, G, tipo):
        H = G.copy()
        if tipo == "aleatorio":
            nodo_remover = random.sample(H.nodes, 1) #Elegir un nodo aleatorio
            #print(nodo_remover)
        elif tipo == "grado":
            max_centrality = max(H.degree, key=lambda x: x[1])[0]
            nodo_remover = [max_centrality]
        elif tipo == "cercania":
            centralidad = nx.closeness_centrality(H) #Hacer la centralidad de cada nodo (diccionario)
        elif tipo == "harmonica":
            centralidad = nx.harmonic_centrality(H) #Hacer la centralidad de cada nodo (diccionario) 
        elif tipo == "pagerank":
            centralidad = nx.pagerank(H) #Por defecto el maximo de iteraciones es 100
        elif tipo == "intermediacion":
            centralidad = nx.betweenness_centrality(H) #Hacer la centralidad de cada nodo (diccionario)
        if tipo == "cercania" or tipo =="harmonica" or tipo == "pagerank" or tipo == "intermediacion":
            max_centrality = (max(centralidad, key=centralidad.get)) #Encontrar la llave con el valor más alto
            nodo_remover = [max_centrality]
        
        H.remove_nodes_from(nodo_remover) #Remover el nodo
        
        return (H, nodo_remover)


    ### FUNCIÓN TIPO DE ATAQUE ###
    def ataque(self, tipo):
        G = self.topologia.copy()
        num_cc_lista = [1] # Número de componentes conectados
        prop_cg_lista = [1] # Proporción del componente gigante
        ataques_lista = [0]
        ataque = 0
        diametro_lista = [self.diametro(G)] # Diámetro        
        longitud_promedio = [self.trayectoria_promedio(G)] # Longitud de la trayectoria Promedio
        probabilidad_conexion = [self.probabilidad_de_conexion(G)] # Probabilidad de conexión de nodos
        nodo_eliminado = [0] #Inicia con 0 nodos

        for x in range(self.num_ataques):
            print("*** Número de ataque: ", x, " ***")
            G, n_eliminado = self.centrality(G, tipo)
            num_cc_lista.append(self.num_componentes_conectados(G))
            prop_cg_lista.append(self.prop_componente_gigante(G))
            ataque = ataque + 1 # ataque += 1
            ataques_lista.append(ataque/len(nx.nodes(self.topologia))) # Proporción de nodos eliminados   
            diametro_lista.append(self.diametro(G)) # Lista Diámetro    
            probabilidad_conexion.append(self.probabilidad_de_conexion(G)) #Lista probabilidad de conexión
            longitud_promedio.append(self.trayectoria_promedio(G)) # Lista Trayectoria Promedio
            nodo_eliminado.append(n_eliminado)

        if tipo == "aleatorio": 
            self.ncc_aleatorio = num_cc_lista
            self.pcg_aleatorio = prop_cg_lista
            self.diametro_aletorio = diametro_lista
            self.aspl_aleatorio = longitud_promedio
            self.pc_aleatorio = probabilidad_conexion
            self.nodo_aleatorio = nodo_eliminado
        
        elif tipo == "grado":
            self.ncc_degree = num_cc_lista
            self.pcg_degree = prop_cg_lista
            self.diametro_degree = diametro_lista
            self.aspl_degree = longitud_promedio
            self.pcg_degree = probabilidad_conexion
            self.nodo_degree = nodo_eliminado
        
        elif tipo == "cercania":
            self.ncc_closeness = num_cc_lista
            self.pcg_closeness = prop_cg_lista
            self.diametro_closeness = diametro_lista
            self.aspl_closeness = longitud_promedio
            self.pc_closeness = probabilidad_conexion
            self.nodo_closeness = nodo_eliminado
        
        elif tipo == "harmonica":
            self.ncc_harmonic = num_cc_lista
            self.pcg_harmonic = prop_cg_lista
            self.diametro_harmonic = diametro_lista
            self.aspl_harmonic = longitud_promedio
            self.pc_harmonic = probabilidad_conexion
            self.nodo_harmonic = nodo_eliminado
        
        elif tipo == "pagerank":
            self.ncc_pagerank = num_cc_lista
            self.pcg_pagerank = prop_cg_lista
            self.diametro_pagerank = diametro_lista
            self.aspl_pagerank = longitud_promedio
            self.pc_pagerank = probabilidad_conexion
            self.nodo_pagerank = nodo_eliminado
        
        elif tipo == "intermediacion":
            self.ncc_betweenness = num_cc_lista
            self.pcg_betweenness = prop_cg_lista
            self.diametro_betweenness = diametro_lista
            self.aspl_betweenness = longitud_promedio
            self.pc_betweenness = probabilidad_conexion
            self.nodo_betweenness = nodo_eliminado
        
        return ataques_lista, num_cc_lista, prop_cg_lista, diametro_lista, longitud_promedio, probabilidad_conexion, nodo_eliminado

In [57]:
def atacar_red(grafo, nombre_grafo, num_ataques):
    r1 = Red(nombre_grafo, grafo, num_ataques)
    # declarar un diccionario por cada metrica
    diametro = dict()
    num_cc = dict()
    proporcion_cg = dict()
    aspl = dict()
    prob_conexion = dict()
    nodo_eliminar = dict()

    tipos_ataques = ["aleatorio", "grado", "cercania", "harmonica", "pagerank", "intermediacion"]
  
    for ataque in tipos_ataques:
        #print("*************************")
        print("Tipo de ataque: " + ataque)
        #imprimir_hora()
        ataques_lista, num_cc_lista, prop_cg_lista, diametro_lista, longitud_promedio, probabilidad_conexion, nodo_eliminado = r1.ataque(ataque)
        #Almacenar las metricas en sus diccionarios
        diametro[ataque] = diametro_lista
        num_cc[ataque] = num_cc_lista
        proporcion_cg[ataque] = prop_cg_lista
        aspl[ataque] = longitud_promedio
        prob_conexion[ataque] = probabilidad_conexion
        nodo_eliminar[ataque] = nodo_eliminado

    #Agregar la lista de ataques a cada diccionario
    diametro["num_ataques"] = ataques_lista
    num_cc["num_ataques"] = ataques_lista
    proporcion_cg["num_ataques"] = ataques_lista
    aspl["num_ataques"] = ataques_lista
    prob_conexion["num_ataques"] = ataques_lista
    nodo_eliminar["num_ataques"] = ataques_lista

    #Almacenar cada diccionario (son 6) en un csv
    pd.DataFrame(diametro).to_csv('resultados\\diametro\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(num_cc).to_csv('resultados\\numero_cc\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(proporcion_cg).to_csv('resultados\\proporcion_cg\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(aspl).to_csv('resultados\\aspl\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(prob_conexion).to_csv('resultados\\prob_conexion\\' + nombre_grafo + '.csv', index=False)
    pd.DataFrame(nodo_eliminar).to_csv('resultados\\nodos_eliminados\\' + nombre_grafo + '.csv', index=False)

In [58]:
folder_resultados = "resultados"
folders = ['diametro', 'numero_cc', 'proporcion_cg', 'aspl', 'prob_conexion', 'nodos_eliminados']

if not os.path.exists(folder_resultados + "\\"):
    os.makedirs(folder_resultados)

for folder in folders:
    if not os.path.exists(folder_resultados + "\\" + folder + "\\"):
        os.makedirs(folder_resultados + "\\" + folder)

In [59]:
for r in (1, 2):
    regla = "r" + str(r)
    for e in (1, 2, 4, 8, 16, 32):
        enlace = "enlaceD" + str(e)

        experimento = "experimento" + str(random.randint(1, 10))
        #experimento = "experimento" + str(11)
        nombre_grafo = regla + "_" + enlace + "_" + experimento + "_ciclo30.graphml"
        ruta_grafo = "graphs\\" + regla + "\\" + enlace + "\\" + experimento + "\\" + nombre_grafo

        grafo = nx.read_graphml(ruta_grafo)
        print("Nombre grafo: " + nombre_grafo)
        print("Ruta grafo: " + ruta_grafo)

        atacar_red(grafo, regla + "_" + enlace + "_" + experimento + "_ciclo30", 3)# 2499)

Nombre grafo: r1_enlaceD1_experimento3_ciclo30.graphml
Ruta grafo: graphs\r1\enlaceD1\experimento3\r1_enlaceD1_experimento3_ciclo30.graphml
Tipo de ataque: aleatorio
*** Número de ataque:  0  ***
Nodos en la red
['1', '2', '51', '601', '13', '3', '52', '552', '101', '61', '451', '4', '53', '803', '11', '102', '602', '62', '5', '54', '504', '103', '553', '753', '6', '55', '805', '855', '104', '64', '754', '56', '7', '756', '9', '105', '67', '57', '106', '706', '63', '8', '857', '907', '58', '10', '12', '107', '757', '59', '108', '758', '1008', '60', '810', '109', '959', '911', '110', '1010', '662', '812', '111', '961', '14', '1113', '1163', '112', '1012', '15', '964', '1014', '113', '863', '1013', '65', '114', '66', '16', '915', '1015', '17', '1116', '1166', '115', '1065', '1115', '18', '967', '1117', '116', '19', '68', '1018', '1168', '117', '1167', '1175', '20', '69', '1069', '1219', '118', '968', '21', '70', '1070', '1220', '119', '1119', '22', '71', '821', '971', '120', '1224', '72'

C:\Users\snake\AppData\Local\Temp\ipykernel_9556\669619608.py:116: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nodo_remover = random.sample(H.nodes, 1) #Elegir un nodo aleatorio


Nombre grafo: r1_enlaceD2_experimento6_ciclo30.graphml
Ruta grafo: graphs\r1\enlaceD2\experimento6\r1_enlaceD2_experimento6_ciclo30.graphml
Tipo de ataque: aleatorio
*** Número de ataque:  0  ***
Nodos en la red
['1', '2', '51', '14', '351', '3', '52', '552', '652', '101', '63', '601', '4', '53', '5', '102', '62', '54', '704', '804', '103', '403', '6', '55', '905', '104', '56', '7', '10', '105', '57', '106', '556', '956', '8', '857', '9', '58', '808', '958', '107', '907', '1207', '59', '909', '959', '108', '11', '60', '810', '12', '109', '709', '61', '961', '1011', '110', '1010', '64', '13', '1062', '111', '911', '863', '913', '112', '1012', '15', '814', '1114', '113', '963', '65', '114', '914', '66', '16', '815', '1115', '17', '1066', '1116', '115', '1165', '18', '67', '1217', '19', '116', '716', '68', '1118', '1218', '117', '367', '1067', '20', '69', '919', '1119', '118', '1224', '21', '70', '1070', '1170', '119', '71', '22', '971', '1271', '120', '72', '23', '922', '1272', '121', '7

C:\Users\snake\AppData\Local\Temp\ipykernel_9556\669619608.py:116: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nodo_remover = random.sample(H.nodes, 1) #Elegir un nodo aleatorio


Nodos en la red
['1', '2', '51', '14', '351', '3', '52', '552', '652', '101', '63', '601', '4', '53', '5', '102', '62', '54', '704', '804', '103', '403', '6', '55', '905', '104', '56', '7', '10', '105', '57', '106', '556', '956', '8', '857', '9', '58', '808', '958', '107', '907', '1207', '59', '909', '959', '108', '11', '60', '810', '12', '109', '709', '61', '961', '1011', '110', '1010', '64', '13', '1062', '111', '911', '863', '913', '112', '1012', '15', '814', '1114', '113', '963', '65', '114', '914', '66', '16', '815', '1115', '17', '1066', '1116', '115', '1165', '18', '67', '1217', '19', '116', '716', '68', '1118', '1218', '117', '367', '1067', '20', '69', '919', '1119', '118', '1224', '21', '70', '1070', '1170', '119', '71', '22', '971', '1271', '120', '72', '23', '922', '1272', '121', '73', '122', '722', '1022', '24', '1023', '1173', '25', '74', '774', '924', '123', '1073', '26', '75', '1075', '1325', '124', '974', '1124', '27', '76', '1026', '1176', '125', '1625', '28', '77', '1

...

Nodos en la red
['1', '2', '51', '10', '501', '3', '52', '602', '101', '66', '4', '53', '603', '102', '652', '5', '54', '504', '9', '103', '6', '55', '855', '104', '64', '604', '56', '7', '806', '105', '655', '57', '106', '906', '58', '8', '657', '857', '158', '458', '107', '59', '659', '759', '108', '61', '11', '60', '810', '12', '109', '861', '13', '110', '62', '312', '862', '111', '65', '14', '63', '813', '863', '112', '15', '164', '864', '113', '67', '114', '564', '664', '16', '715', '765', '17', '616', '766', '115', '915', '18', '167', '817', '116', '216', '366', '19', '68', '168', '468', '117', '217', '717', '20', '69', '119', '169', '118', '21', '70', '170', '320', '919', '71', '22', '321', '771', '120', '220', '370', '72', '23', '172', '322', '121', '521', '821', '73', '122', '222', '522', '24', '323', '773', '25', '74', '174', '874', '123', '523', '823', '26', '75', '625', '775', '124', '524', '824', '27', '76', '476', '626', '125', '375', '825', '28', '77', '327', '627', '126

Nombre grafo: r1_enlaceD8_experimento6_ciclo30.graphml
Ruta grafo: graphs\r1\enlaceD8\experimento6\r1_enlaceD8_experimento6_ciclo30.graphml
Tipo de ataque: aleatorio
*** Número de ataque:  0  ***
Nodos en la red
['1', '2', '51', '9', '401', '3', '52', '10', '152', '101', '451', '58', '4', '53', '403', '102', '5', '54', '404', '103', '61', '6', '55', '205', '104', '154', '204', '56', '7', '13', '156', '105', '57', '106', '8', '357', '258', '308', '107', '65', '307', '59', '359', '409', '108', '458', '11', '60', '15', '360', '109', '459', '64', '12', '211', '110', '460', '62', '362', '412', '111', '14', '63', '313', '112', '462', '314', '414', '113', '213', '363', '114', '364', '464', '16', '215', '365', '17', '66', '116', '166', '115', '18', '67', '117', '167', '266', '416', '19', '68', '168', '318', '367', '417', '20', '69', '169', '319', '118', '218', '368', '21', '70', '170', '320', '119', '219', '22', '71', '121', '171', '120', '220', '72', '23', '172', '322', '221', '371', '73', '1

C:\Users\snake\AppData\Local\Temp\ipykernel_9556\669619608.py:116: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nodo_remover = random.sample(H.nodes, 1) #Elegir un nodo aleatorio


Nodos en la red
['1', '2', '51', '9', '401', '3', '52', '10', '152', '101', '451', '58', '4', '53', '403', '102', '5', '54', '404', '103', '61', '6', '55', '205', '104', '154', '204', '56', '7', '13', '156', '105', '57', '106', '8', '357', '258', '308', '107', '65', '307', '59', '359', '409', '108', '458', '11', '60', '15', '360', '109', '459', '64', '12', '211', '110', '460', '62', '362', '412', '111', '14', '63', '313', '112', '462', '314', '414', '113', '213', '363', '114', '364', '464', '16', '215', '365', '17', '66', '116', '166', '115', '18', '67', '117', '167', '266', '416', '19', '68', '168', '318', '367', '417', '20', '69', '169', '319', '118', '218', '368', '21', '70', '170', '320', '119', '219', '22', '71', '121', '171', '120', '220', '72', '23', '172', '322', '221', '371', '73', '122', '222', '372', '24', '173', '323', '25', '74', '174', '324', '123', '223', '373', '26', '75', '175', '325', '124', '224', '374', '27', '76', '126', '176', '125', '225', '375', '28', '77', '177

KeyboardInterrupt: 

In [61]:
grafo = nx.path_graph(5)
atacar_red(grafo, "prueba", 4)# 2499)

Tipo de ataque: aleatorio
*** Número de ataque:  0  ***
Nodos en la red
[0, 1, 2, 3, 4]
*** Número de ataque:  1  ***
Nodos en la red
[0, 1, 2, 3]
*** Número de ataque:  2  ***
Nodos en la red
[0, 1, 3]
*** Número de ataque:  3  ***
Nodos en la red
[1, 3]


C:\Users\snake\AppData\Local\Temp\ipykernel_9556\669619608.py:116: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nodo_remover = random.sample(H.nodes, 1) #Elegir un nodo aleatorio
